Now let's fit all of this into a Bert model

In [ ]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, AutoTokenizer
from torch.utils.data import Dataset, random_split
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import ast
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

Getting the tokens and the sentences

In [ ]:
df = pd.read_csv("/content/filtered_sentences_tags.csv")
print(df['Tokens'].head())
token_lists = df['Tokens'].apply(ast.literal_eval).tolist()
tags = df['Tagged Sentence'].apply(ast.literal_eval).tolist()

print(token_lists[:5])
print(tags[:5])

Functions to prepare the dataset for our model, to get train/val/test sets and to tokenize the sentences specifically for BERT. Here we also change annotation to int for the model

In [ ]:
def split_dataset(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1"

    total_size = len(dataset)
    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)
    test_size = total_size - train_size - val_size

    return random_split(dataset, [train_size, val_size, test_size])

#Loading pre-trained tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tag2id = {'O': 0, 'B-MOUNTAIN': 1, 'I-MOUNTAIN': 2}
id2tag = {id: tag for tag, id in tag2id.items()}

#Tokenizing and align labels
def tokenize_and_align_labels(token_lists, tags):
    tokenized_inputs = tokenizer(token_lists, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(tags):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(tag2id[label[word_idx]] if label[word_idx] == 'I-MOUNTAIN' else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Create class for dataset
class NERDataset(Dataset):
    def __init__(self, tokenized_inputs):
        self.tokenized_inputs = tokenized_inputs
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.tokenized_inputs.items()}
    def __len__(self):
        return len(self.tokenized_inputs.input_ids)

Aaand the dataset, we save the test set for future inference

In [ ]:
# Tokenize and create dataset
tokenized_inputs = tokenize_and_align_labels(token_lists, tags)
full_dataset = NERDataset(tokenized_inputs)
train_dataset, val_dataset, test_dataset = split_dataset(full_dataset)
test_dataset.to_csv("/content/test_data.csv")

A function for evaluating model performance, regular precision, recall and accuracy

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Remove ignored index (special tokens)
    true_labels = [[l for l in label if l != -100] for label in labels]
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    all_true_labels = sum(true_labels, [])
    all_true_predictions = sum(true_predictions, [])

    precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_true_predictions, average='weighted')
    acc = accuracy_score(all_true_labels, all_true_predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


Model definition, decided to go with 3 epochs and default everything else, turns out it is more than enough

In [ ]:
#Had to create this function for padding, there's probably a better way
def custom_collate(batch):
    batch_dict = {}
    for key in batch[0].keys():
        if key == 'labels':
            batch_dict[key] = pad_sequence([item[key] for item in batch], batch_first=True, padding_value=-100)
        else:
            batch_dict[key] = pad_sequence([item[key] for item in batch], batch_first=True, padding_value=0)

    return batch_dict

model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(tag2id))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=1000,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=custom_collate,
    compute_metrics=compute_metrics,
)


trainer.train()

# Save the model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")
print("Model training completed and saved.")

test_results = trainer.evaluate(test_dataset)
print("Test set results:", test_results)

# Save the model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")
print("Model training completed and saved.")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,0.515300,0.443215,0.878100,0.821106,0.771060,0.878100
100,0.206300,0.162381,0.944725,0.943493,0.942901,0.944725
150,0.115600,0.103574,0.957310,0.959240,0.963120,0.957310
200,0.097500,0.088467,0.961382,0.963430,0.967820,0.961382
250,0.042600,0.077351,0.971252,0.971343,0.971568,0.971252
300,0.061500,0.067729,0.973720,0.974134,0.974792,0.973720


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model training completed and saved.


Test set results: {'eval_loss': 0.06276286393404007, 'eval_accuracy': 0.978204010462075, 'eval_f1': 0.9786146294465117, 'eval_precision': 0.9794073285621135, 'eval_recall': 0.978204010462075, 'eval_runtime': 72.6178, 'eval_samples_per_second': 5.15, 'eval_steps_per_second': 0.083, 'epoch': 3.0}
Model training completed and saved.


Training process went pretty neatly, has good metrics on train and validation sets

In [ ]:
def perform_ner(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=2)

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    ner_results = []
    for token, prediction in zip(tokens, predictions[0]):
        if prediction != 0:  # 0 is the ID for 'O'
            ner_results.append((token, id2tag[prediction.item()]))

    return ner_results
# Test the model
test_sentence = "I like to walk and run"
results = perform_ner(test_sentence)
print(f"NER results for '{test_sentence}':")
print(results)

NER results for 'I like to walk and run':
[]
